In [1]:
%cd /app

/app


In [2]:
import argparse
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import torch
torch.multiprocessing.set_start_method('spawn')

import jax
from lob.encoding import Vocab, Message_Tokenizer

from lob import inference_no_errcorr as inference
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint

from lob import inference_no_errcorr as inference
import lob.encoding as encoding
import preproc as preproc

import jax.numpy as jnp
import numpy as onp

from pathlib import Path
import os

2025-03-12 22:33:30.094854: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
save_folder = 'data_saved/'
batch_size =4
model_size = 'large'
data_dir ='data/test_set/'
rng_seed = 42
sample_all = False # action='store_true'

In [4]:
stock = 'GOOG'  # 'GOOG', 'INTC'
n_gen_msgs = 500 # how many messages to generate into the future
n_samples = 32
tick_size = 100

In [5]:
n_messages = 500  # length of input sequence to model
# batch_size = batch_size

v = Vocab()
n_classes = len(v)
seq_len = n_messages * Message_Tokenizer.MSG_LEN
book_dim = 501 #b_enc.shape[1]
book_seq_len = n_messages

n_eval_messages = n_gen_msgs  # how many to load from dataset
eval_seq_len = n_eval_messages * Message_Tokenizer.MSG_LEN

rng = jax.random.key(rng_seed)
rng, rng_ = jax.random.split(rng)

In [6]:
if stock == 'GOOG':
    # ckpt_path = './checkpoints/treasured-leaf-149_84yhvzjt/' # 0.5 y GOOG, (full model)
    ckpt_path = './checkpoints/denim-elevator-754_czg1ss71/' # large model
    # ckpt_path = './checkpoints/stilted-deluge-759_8g3vqor4'  # small model
elif stock == 'INTC':
    # ckpt_path = './checkpoints/pleasant-cherry-152_i6h5n74c/' # 0.5 y INTC, (full model)
    ckpt_path = './checkpoints/eager-sea-755_2rw1ofs3/'  # large model
else:
    raise ValueError(f'stock {stock} not recognized')

In [7]:
print('Loading metadata:', ckpt_path)
args_ckpt = load_metadata(ckpt_path)

Loading metadata: ./checkpoints/denim-elevator-754_czg1ss71/


In [8]:
# scale down to single GPU, single sample inference
args_ckpt.bsz = 1 #1, 10
args_ckpt.num_devices = 1

batchnorm = args_ckpt.batchnorm

In [9]:
# load train state from disk

print('Initializing model...')
new_train_state, model_cls = init_train_state(
    args_ckpt,
    n_classes=n_classes,
    seq_len=seq_len,
    book_dim=book_dim,
    book_seq_len=book_seq_len,
)

print('\nLoading model checkpoint...')
ckpt = load_checkpoint(
    new_train_state,
    ckpt_path,
    train=False,
)
state = ckpt['model']

model = model_cls(training=False, step_rescale=1.0)

param_count = sum(x.size for x in jax.tree_leaves(state.params))
print('param count:', param_count)

Initializing model...
configuring standard optimization setup
[*] Trainable Parameters: 35776312

Loading model checkpoint...
param count: 35776312


/tmp/ipykernel_278913/2594640255.py:22: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  param_count = sum(x.size for x in jax.tree_leaves(state.params))


In [10]:
data_dir = data_dir + stock
# data_dir = data_dir + 'GOOG'
print(f"Directory Path: {data_dir}")

Directory Path: data/test_set/GOOG


In [11]:
# data_dir = Path.home() / "data/test_set/GOOG"
# Or use a relative path from current working directory
# data_dir = Path("data/test_set/GOOG")
data_dir = Path(data_dir)

Path(data_dir).mkdir(parents=True, exist_ok=True)
folder_path = Path(data_dir)
file_count = len([f for f in folder_path.iterdir() if f.is_file()])
print(f"There are {file_count} files in the folder {str(data_dir)}.")

There are 36 files in the folder data/test_set/GOOG.


In [12]:
print(n_messages, n_eval_messages)

500 500


In [13]:
from pathlib import Path
import os

# Print current working directory to help verify the path
print(f"Current working directory: {os.getcwd()}")

# Use relative path to data/test_set/GOOG
data_dir = Path("data/test_set/GOOG")

try:
    Path(data_dir).mkdir(parents=True, exist_ok=True)
    print(f"Successfully created or verified directory: {data_dir}")
    
    file_count = len([f for f in Path(data_dir).iterdir() if f.is_file()])
    print(f"There are {file_count} files in the folder {data_dir}.")
except Exception as e:
    print(f"Error: {str(e)}")

ds = inference.get_dataset(data_dir, n_messages, n_eval_messages)

Current working directory: /app
Successfully created or verified directory: data/test_set/GOOG
There are 36 files in the folder data/test_set/GOOG.


# check 1 by 1

In [14]:
batch_i = [4497, 6855]
m_seq, _, b_seq_pv, msg_seq_raw, book_l2_init = ds[batch_i]

# i_sample = 4497
# m_seq, _, b_seq_pv, msg_seq_raw, book_l2_init = ds[i_sample]


print(f'm_seq: {m_seq}\n\n')
print(f'b_seq_pv: {b_seq_pv}\n\n')
print(f'msg_seq_raw: {msg_seq_raw}\n\n')
print(f'book_l2_init: {book_l2_init}\n\n')

m_seq: (Array([ 1005, 12010, 12007, ...,   858,   838,   719], dtype=int32), Array([ 1003, 12009, 12008, ...,     2,     2,     2], dtype=int32))


b_seq_pv: (memmap([[     0, 884500,    225, ...,    707, 883400,    200],
        [     0, 884500,    225, ...,    707, 883400,    200],
        [     0, 884500,    225, ...,    707, 883400,    200],
        ...,
        [     0, 884600,    325, ...,    200, 883500,    100],
        [     0, 884600,    325, ...,    200, 883500,    100],
        [     0, 884600,    325, ...,    200, 883500,    100]]), memmap([[     0, 874900,    179, ...,   1297, 873900,    841],
        [     0, 874900,    179, ...,   1297, 873900,    841],
        [     0, 874900,    179, ...,   1297, 873900,    841],
        ...,
        [     0, 874700,    200, ...,    889, 873700,    947],
        [     0, 874700,    100, ...,    889, 873700,    947],
        [     0, 874700,    100, ...,    889, 873700,    947]]))


msg_seq_raw: (Array([[815620622,         3,         1

In [15]:
m_seq_gen_doubled, b_seq_gen_doubled, msgs_decoded_doubled, l2_book_states_halved, sim_init, sim_states_init = inference.run_generation_scenario(
    n_samples, 
    batch_size,
    ds,
    rng,
    seq_len,
    n_messages,
    n_gen_msgs,
    state,
    model,
    batchnorm,
    v.ENCODING,
    stock,
    n_vol_series = 500,
    # sim_book_levels: int = 20,
    # sim_queue_len: int = 100,
    # data_levels: int = 10,
    save_folder = './data_saved/',
    tick_size = 100,
    sample_top_n = -1,
    sample_all = False,
    # Insertions variables
    num_insertions = 20,
    num_coolings = 20
    )

  0%|          | 0/2 [00:00<?, ?it/s]

BATCH [4497, 6855]

ITERATION  1
1: book_l2_init shape: (2, 40):
[[884500    225 884300    400 884600    225 884200    250 884700    353
  884100    625 884800    349 884000    325 884900    925 883900    625
  885000   1900 883800   1122 885100    300 883700    100 885200    300
  883600    100 885300    300 883500    100 885400    707 883400    200]
 [874900    179 874800    381 875000   1168 874700    482 875100   1738
  874600    427 875200   1488 874500   1182 875300    510 874400   1086
  875400   1100 874300    983 875500   1050 874200    400 875600    910
  874100    250 875700    445 874000    557 875800   1297 873900    841]]

1: m_seq_raw_inp shape: (2, 500, 14):
[[[815620622         3         1 ...       100     53060 276711069]
  [815626126         1         1 ...     -9999     -9999     -9999]
  [815628422         1         0 ...     -9999     -9999     -9999]
  ...
  [815716726         1         0 ...     -9999     -9999     -9999]
  [815716734         1         0 ...   

/opt/conda/envs/myenv/lib/python3.12/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


In [ ]:
pwd

# Order insertion

What do we receive from last generation:

1. m_seq_gen_doubled, 

2. b_seq_gen_doubled, 

3. msgs_decoded_doubled, 

4. l2_book_states_halved, 

5. sim_init, 

6. sim_states_init

In [ ]:
# msgs_decoded_doubled.shape

In [ ]:
# TIMEs_i = msgs_decoded_doubled[:, -1:, 8]
# TIMEns_i = msgs_decoded_doubled[:, -1:, 9]

# tick_size = 100

# # mid_price = (l2_book_states_halved[0][0].item() + l2_book_states_halved[0][2].item()) / 2
# mid_price = (l2_book_states_halved[:, 0] + l2_book_states_halved[:, 2]) / 2
# mid_price = mid_price[:, None]

# ORDER_ID_i = 501
# EVENT_TYPE_i = 4
# DIRECTION_i = 1
# PRICE_i = mid_price
# SIZE_i = 75
# DTs_i = 0
# DTns_i = 0

# PRICE_REF_i = 0
# SIZE_REF_i = 0
# TIMEs_REF_i = 0
# TIMEns_REF_i = 0

# p_abs = mid_price + PRICE_REF_i * tick_size
# PRICE_ABS_i = 3

# batch_size = TIMEns_i.shape[0]

# # 1
# ins_msg = jnp.concatenate([
#     jnp.full((batch_size, 1), ORDER_ID_i),
#     jnp.full((batch_size, 1), EVENT_TYPE_i),
#     jnp.full((batch_size, 1), DIRECTION_i),
#     mid_price,
#     PRICE_i,
#     jnp.full((batch_size, 1), SIZE_i),
#     jnp.full((batch_size, 1), DTs_i),
#     jnp.full((batch_size, 1), DTns_i),
#     TIMEs_i,
#     TIMEns_i,
#     jnp.full((batch_size, 1), PRICE_REF_i),
#     jnp.full((batch_size, 1), SIZE_REF_i),
#     jnp.full((batch_size, 1), TIMEs_REF_i),
#     jnp.full((batch_size, 1), TIMEns_REF_i)
# ], axis=1)

# print("Batched insertion message shape:", ins_msg.shape)  # Expected: (batch_size, 14)
# print(type(ins_msg))
# print(ins_msg)

In [ ]:
# msg_decoded = ins_msg  # assign for clarity
# # print(f"msg_deco|ded shape: {msg_decoded.shape}: {msg_decoded}\n")

# # Let's assume new_part is the first N_NEW_FIELDS tokens (you can adjust as needed)
# new_part = msg_decoded#[:, :Message_Tokenizer.N_NEW_FIELDS]  # shape (batch_size, N_NEW_FIELDS)


# batched_side = jnp.array([DIRECTION_i] * batch_size)
# batched_time_s_ref = jnp.array([TIMEs_REF_i] * batch_size)  
# batched_time_ns_ref = jnp.array([TIMEns_REF_i] * batch_size)
# batched_EVENT_TYPE = jnp.array([EVENT_TYPE_i] * batch_size)
# batched_quantity = jnp.array([SIZE_i] * batch_size)
# batched_p_abs = p_abs
# new_order_id = 999
# batched_new_order_id = jnp.array([new_order_id] * batch_size)


# # 2
# batched_get_order = jax.vmap(sim_init.get_order_at_time, in_axes=(0, 0, 0, 0))
# orig_orders = batched_get_order(sim_states_init, batched_side, batched_time_s_ref, batched_time_ns_ref) # DO WE ACTUALLY RECEIVE WHAT WE NEED?
# # print(f"orig_orders: {orig_orders}\n")
# order_id_ref = orig_orders[:, 2]  # shape (batch_size,)
# # print(f"order_id_ref: {order_id_ref}\n")
# order_id = jnp.where((batched_EVENT_TYPE == 2) | (batched_EVENT_TYPE == 3),
#                      order_id_ref,
#                      batched_new_order_id)  # shape (batch_size,)
# # print(f"order_id: {order_id}\n")




# batched_EVENT_TYPE = batched_EVENT_TYPE[:, None].squeeze(-1)
# batched_side = batched_side[:, None].squeeze(-1)
# batched_quantity = batched_quantity[:, None].squeeze(-1)
# order_id = order_id[:, None].squeeze(-1)
# batched_p_abs = batched_p_abs.squeeze(-1)
# batched_time_s = TIMEs_i.squeeze(-1)
# batched_time_ns = TIMEns_i.squeeze(-1)



# # print(f"batched_EVENT_TYPE shape: {batched_EVENT_TYPE.shape}: {batched_EVENT_TYPE}\n")
# # print(f"batched_side shape: {batched_side.shape}: {batched_side}\n")
# # print(f"batched_quantity shape: {batched_quantity.shape}: {batched_quantity}\n")
# # print(f"batched_p_abs shape: {batched_p_abs.shape}: {batched_p_abs}\n")
# # print(f"order_id shape: {order_id.shape}: {order_id}\n")
# # print(f"batched_time_s shape: {TIMEs_i.shape}: {TIMEs_i}\n")
# # print(f"batched_time_ns shape: {TIMEns_i.shape}: {TIMEns_i}\n")



# # 3
# batched_construct_sim_msg = jax.vmap(inference.construct_sim_msg)

# batched_sim_msg = batched_construct_sim_msg(
#     batched_EVENT_TYPE,
#     batched_side,
#     batched_quantity,
#     batched_p_abs,
#     order_id,
#     batched_time_s,
#     batched_time_ns,
# )


# # print(f"msg_decoded shape: {batched_sim_msg.shape}: {batched_sim_msg}\n")




# # 4

# msg_decoded = msg_decoded.at[:, PRICE_ABS_i].set(batched_p_abs) \
#                          .at[:, ORDER_ID_i].set(order_id)


# dummy_msg_single = inference.construct_dummy_sim_msg()
# dummy_msg = jnp.tile(dummy_msg_single, (batch_size, 1))


# sim_msg, msg_decoded = jax.lax.cond(
#     jnp.isnan(new_part).any(),
#     lambda _: (dummy_msg, msg_decoded),
#     lambda _: (batched_sim_msg, msg_decoded),
#     operand=None
# )


# print(f"sim_msg shape: {sim_msg.shape}: {sim_msg}\n")
# print(f"msg_decoded shape: {msg_decoded.shape}: {msg_decoded}\n")

# msg_encoded = jax.vmap(lambda m: encoding.encode_msg(m, v.ENCODING))(msg_decoded)


# print(f"msg_encoded shape: {msg_encoded.shape}: {msg_encoded}\n")
# print(f"m_seq_gen_doubled shape: {m_seq_gen_doubled.shape}: {m_seq_gen_doubled}\n")

# shift = msg_encoded.shape[1]
# UPDATED_m_seq_gen_doubled = jnp.concatenate([m_seq_gen_doubled[:, shift:], msg_encoded], axis=1)


# shift = msg_decoded[:, None, :].shape[1]
# UPDATED_msgs_decoded_doubled = jnp.concatenate(
#     [msgs_decoded_doubled[:, shift:, :], msg_decoded[:, None, :]],
#     axis=1
# )

# print(f"UPDATED_m_seq_gen_doubled AFTER ADDING TO THE END shape: {UPDATED_m_seq_gen_doubled.shape}: {UPDATED_m_seq_gen_doubled}\n")
# print(f"UPDATED_msgs_decoded_doubled AFTER ADDING TO THE END shape: {UPDATED_msgs_decoded_doubled.shape}: {UPDATED_msgs_decoded_doubled}\n")

What do we receive from last generation:

1. m_seq_gen_doubled - DONE

2. b_seq_gen_doubled, - DONE

3. msgs_decoded_doubled, - DONE

4. l2_book_states_halved, - DONE

5. sim_init, 

6. sim_states_init

In [ ]:
# print(f"b_seq_gen_doubled shape: {b_seq_gen_doubled.shape}: {b_seq_gen_doubled}\n")
# print(f"l2_book_states_halved shape: {l2_book_states_halved.shape}: {l2_book_states_halved}\n")

# l2_state_n = 20

# new_sim_state = jax.vmap(sim_init.process_order_array)(sim_states_init, sim_msg)
# new_b_state = jax.vmap(sim_init.get_L2_state, in_axes=(0, None))(new_sim_state, l2_state_n)
# print(f"new_b_state shape before adding time axis: {new_b_state.shape}: {new_b_state}\n")



# new_b_state = new_b_state[:, None, :]
# print(f"new_b_state shape after adding time axis: {new_b_state.shape}: {new_b_state}\n")


# new_state_slice = new_b_state[:, 0, :]
# print(f"new_state_slice shape: {new_state_slice.shape}: {new_state_slice}\n")

# old_last_time_step = b_seq_gen_doubled[:, -1, :]
# print(f"old_last_time_step shape: {old_last_time_step.shape}: {old_last_time_step}\n")


# # why b_seq_gen_doubled shape: (2, 1000, 501): - why I only refresh first 80 levels - why book has 501 level??
# updated_last_time_step = jnp.concatenate([new_state_slice, old_last_time_step[:, 80:]], axis=1)
# print(f"updated_last_time_step shape: {updated_last_time_step.shape}: {updated_last_time_step}\n")



# new_b_seq = jnp.concatenate([b_seq_gen_doubled[:, 1:, :], updated_last_time_step[:, None, :]], axis=1)
# print(f"new_b_seq shape: {new_b_seq.shape}: {new_b_seq}\n")

# new_l2_book_states_halved = updated_last_time_step[:, :40]
# print(f"new_l2_book_states_halved shape: {new_l2_book_states_halved.shape}: {new_l2_book_states_halved}\n")


In [21]:
# a,b,c,d = inference.insert_custom_end(m_seq_gen_doubled, b_seq_gen_doubled, msgs_decoded_doubled, l2_book_states_halved, sim_init, sim_states_init, v.ENCODING)